In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [2]:
lr = 1e-3
batch_size = 64
num_epoch = 3

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
"""
    determine size = [(W−K+2P)/S]+1
    EX)
    W: input=256
    K: kernel_size=4
    P: padding=1
    S: stride=2
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
            # size = [(28 - 5 + 0) / 1] + 1 = 24
        self.pool1 = nn.MaxPool2d(kernel_size=2)
            # after maxpooling size = [(24 + 0 - 2) / 2] + 1 = 12
        self.leakyRelu1 = nn.LeakyReLU()
        self.conv2 = nn.Conv2d(in_channels=10,out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
            # size = [(12 - 5 + 0) / 1] + 1 = 8
        self.pool2 = nn.MaxPool2d(kernel_size=2)
            # after maxpooling size = [(8 + 0 - 2) / 2] + 1 = 4
        self.leakyRelu2 = nn.LeakyReLU()
        
        self.fc1 = nn.Linear(in_features=4*4*20, out_features=50, bias=True)
        self.leakyRelu_fc1 = nn.LeakyReLU()
        
        self.fc2 = nn.Linear(in_features=50, out_features=10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.leakyRelu1(x)
        
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.leakyRelu2(x)
        
        x = x.view(-1,320)
        
        x = self.fc1(x)
        x = self.leakyRelu_fc1(x)
        x = self.fc2(x)
        
        return x

In [5]:
transform = transforms.Compose({
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
})

dataset = datasets.MNIST(download=True, root='./', train=True, transform=transform)
loader = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=0)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729141890/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
print(dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )


In [7]:
num_data = len(loader)
num_batch = np.ceil(num_data / batch_size)

In [8]:
net = Net().to(device)
params = net.parameters()

In [9]:
fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

In [10]:
for epoch in range(1, num_epoch + 1):
    net.train()
    
    loss_arr = []
    acc_arr = []
    
    for batch, (input, label) in enumerate(loader,1):
        input = input.to(device)
        label = label.to(device)
        
        output = net(input)
        pred = fn_pred(output)
        
        optim.zero_grad()
        
        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)
        
        loss.backward()
        
        optim.step()
        
        loss_arr += [loss.item()]
        acc_arr += [acc.item()]
        
        if batch % 100 == 0:
            print(f'{epoch}/{num_epoch} :: Loss : {np.mean(loss_arr)} Acc : {np.mean(acc_arr)}')
        
        

1/5 :: Loss : 0.9576043872535229 Acc : 0.73078125
1/5 :: Loss : 0.603386782258749 Acc : 0.82609375
1/5 :: Loss : 0.4603652273366849 Acc : 0.86640625
1/5 :: Loss : 0.37915472947061063 Acc : 0.88984375
1/5 :: Loss : 0.3264328570403159 Acc : 0.90496875
1/5 :: Loss : 0.2887494797787319 Acc : 0.916015625
1/5 :: Loss : 0.2597648823833359 Acc : 0.9241517857142857
1/5 :: Loss : 0.23756196094094775 Acc : 0.93046875
1/5 :: Loss : 0.2209395698033687 Acc : 0.9349479166666667
2/5 :: Loss : 0.06345479793380945 Acc : 0.9815625
2/5 :: Loss : 0.06346665801713243 Acc : 0.98078125
2/5 :: Loss : 0.06621629701151202 Acc : 0.9798958333333333


KeyboardInterrupt: 